In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, f1_score
from sklearn.metrics import 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from itertools import product
import os
from tensorflow.keras.models import load_model
import random
import openpyxl  
from tensorflow.keras import layers, Sequential



In [ ]:
data = pd.read_csv('ModeChoiceOptima.txt', sep="\t", header=None)

# Renaming correctly the columns of pandas data
columnsID = data.columns
featureNames = data.iloc[0, :]
NameDictionary = dict(zip(columnsID, featureNames))
data.rename(columns = NameDictionary, inplace = True)
data.drop(index=data.index[0], axis=0, inplace=True)

data.drop((data[data.Choice == '-1']).index, inplace = True)
data.drop((data[data.Choice == '-2']).index, inplace = True)
data.drop((data[data.Choice == '2']).index, inplace = True)

# Replacing all other '-1' and '-2' with NaN for consistency
data.replace('-1', np.nan, inplace=True)
data.replace('-2', np.nan, inplace=True)

# Changing the dtype to numerical values, because it should help speed things up
data = data.apply(pd.to_numeric, errors='raise')

DelColumns = ['ID', 'Weight', 'CostCar', 'CoderegionCAR']
data.drop(columns=DelColumns, inplace=True)
print("data shape: ", data.shape)

ReqDummy = ['DestAct', 'HouseType', 'Mothertongue', 'FamilSitu', 'OccupStat', 'SocioProfCat', 'Education', 'TripPurpose', 'TypeCommune', 'ClassifCodeLine', 'ResidChild', 'Region', 'ModeToSchool']

dummydata = pd.DataFrame()
for feat in ReqDummy:
    if feat in data.columns:
        dummydata[feat] = data[feat]
    else:
        print(f"Column '{feat}' not found in data")
    
    # Deleting the original column in the main dataframe
    data.drop(feat, axis = 1, inplace = True)

y = data['Choice']
X = data.drop(columns=['Choice'])
X_dummy = dummydata

print("X shape: ", X.shape)
print("X_dummy shape: ", X_dummy.shape)
print("y ", y.shape)

In [ ]:
# Standardizing the data (ONLY X)

scaler = StandardScaler()
temp_scaled = scaler.fit_transform(X)

# create a new DataFrame with the scaled data
X_scaled = pd.DataFrame(temp_scaled, columns=X.columns)
print('X_scaled shape: ', X_scaled.shape)
print('X_dummy shape: ', X_dummy.shape)

# Checking that the dimensions are right and amount of NaN entries
y = y.reset_index(drop=True)
X_scaled = X_scaled.reset_index(drop=True)
X_dummy = X_dummy.reset_index(drop=True)
X_tot = pd.concat([X_scaled, X_dummy], axis=1)
y_standard = y
X_scaled_standard = X_scaled
X_dummy_standard = X_dummy
X_tot_standard = X_tot


In [ ]:
def dummy_inator(X, na_indicator = True):
    # X is the input dataframe with all the columns that need to be split into dubby variables
    # na_indicator is a boolean variable (optional function input)
    # if na_indicator is False, NaN entries will lead to all False dummy variables
    # if na_indicator is True, an extra dummy variable will be created for each feature, indicating (True/False) if there was a NaN entry
    print('----- Generating the dummy variables')
    DumbOutput = pd.DataFrame()
    columns = X.columns
    for feat in columns:
        dummies = pd.get_dummies(X[feat], prefix=feat, dummy_na=na_indicator)
        DumbOutput = pd.concat([DumbOutput, dummies], axis=1)
        print(f"fixed {feat} by adding {dummies.shape[1]} dummy variables.")
    print('-----')
    return DumbOutput

In [ ]:
def evaluate_model(model, X_test, y_test):

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Convert predictions from one-hot encoding to class labels
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Generate classification report with F1-score
    report = classification_report(y_test_classes, y_pred_classes, digits=4)

    print("Classification Report:\n", report)

def train_model(model, X_train, X_test, y_train, y_test, epochs, batch_size, results_show=False):
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=0)

    if results_show:
        evaluate_model(model, X_test, y_test)

    return history

def create_model(X_train, X_test, y_train, y_test, results_show=False):
    model = keras.Sequential([
        layers.Input(shape=(X_train.shape[1],) ),  # Input layer with 52 neurons
        layers.Dense(64, activation='relu'),  # First hidden layer
        layers.Dense(32, activation='relu'),  # Second hidden layer
        layers.Dense(16, activation='relu'),  # Third hidden layer
        layers.Dense(2, activation='softmax')  # Output layer with 2 neurons (classification)
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    if results_show:
        model.summary()
    
    return model

def plot_history(history):
    # Plot training & validation accuracy
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
def augment_data(X, y, Tsize=0.2):
    # Check class distribution before SMOTE
    unique, counts = np.unique(y, return_counts=True)
    print("Before SMOTE:", dict(zip(unique, counts)))

    # Split the data into training and testing sets (use original y here)
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=Tsize, random_state=RandomState, stratify=y)

    # Apply SMOTE to integer labels
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X, y)

    # Check class distribution after SMOTE
    unique, counts = np.unique(y_train_resampled, return_counts=True)
    print("After SMOTE:", dict(zip(unique, counts)))

    return X_train_resampled, y_train_resampled

# Making the data split before imputation and/or augmentation
def make_custom_split (X_scaled, X_dummy, y, valid_rows, random_state):
    # Identify valid rows (no NaNs in either dataset)
    #valid_rows = (~X_scaled.isna().any(axis=1)) & (~X_dummy.isna().any(axis=1))
    
    # Create clean versions
    X_scaled_clean = X_scaled[valid_rows]
    X_dummy_clean = X_dummy[valid_rows]
    y_clean = y[valid_rows]
    
    # Train-test split using indices
    X_indices = X_scaled_clean.index  # Save original indices
    train_idx, test_idx = train_test_split(X_indices, random_state=random_state, test_size=0.2)
    
    # Create train/test splits
    X_train_scaled = X_scaled.loc[train_idx]
    X_train_dummy  = X_dummy.loc[train_idx]
    y_train        = y.loc[train_idx]
    # these train datapoints will not actually get used, because we want to generate the train data after imputation and augmentation...
    
    # the test datapoints are good and will be used
    X_test_scaled  = X_scaled.loc[test_idx]
    X_test_dummy   = X_dummy.loc[test_idx]
    y_test         = y.loc[test_idx]
    
    # Update the original X_scaled and X_dummy to exclude the test rows
    X_scaled = X_scaled.drop(index=test_idx)
    X_dummy  = X_dummy.drop(index=test_idx)
    y = y.drop(index = test_idx)

    return X_test_scaled, X_test_dummy, y_test, X_scaled, X_dummy, y

In [ ]:
def run_pipeline(Filltype, dataAugmentation, CatDealer, use_attitudes, X_dummy, X_scaled, y, seed, return_data=False):
    if CatDealer:
        print('Deleting rows with NaNs categorical features')
        # Identify rows in X_dummy that contain NaN values
        nan_mask = X_dummy.isna().any(axis=1)  # True for rows with any NaN
        # Drop those rows in both X_dummy, X_scaled and y
        X_dummy_cleaned = X_dummy[~nan_mask].reset_index(drop=True)
        X_scaled_cleaned = X_scaled[~nan_mask].reset_index(drop=True)
        y_cleaned = y[~nan_mask].reset_index(drop=True)
        print('y shape: ', y_cleaned.shape)
        print('X_scaled shape: ', X_scaled_cleaned.shape)
        print('X_dummy shape: ', X_dummy_cleaned.shape)
        X_WithDummies = dummy_inator(X_dummy_cleaned, False)

        # Identify valid rows (no NaNs in either dataset)
        valid_rows = (~X_scaled_cleaned.isna().any(axis=1)) & (~X_dummy_cleaned.isna().any(axis=1))
        X_test_scaled, X_test_dummy, y_test, X_scaled, X_dummy, y = make_custom_split(X_scaled_cleaned, X_WithDummies, y_cleaned, valid_rows, seed)

    else:
        print(X_dummy)
        X_dummy_cleaned = X_dummy
        X_scaled_cleaned = X_scaled
        y_cleaned = y
        print('y shape: ', y_cleaned.shape)
        print('X_scaled shape: ', X_scaled_cleaned.shape)
        print('X_dummy shape: ', X_dummy_cleaned.shape)
        X_WithDummies = dummy_inator(X_dummy_cleaned, True)

        # Identify valid rows (no NaNs in either dataset)
        valid_rows = (~X_scaled_cleaned.isna().any(axis=1)) & (~X_dummy_cleaned.isna().any(axis=1))
        X_test_scaled, X_test_dummy, y_test, X_scaled, X_dummy, y = make_custom_split(X_scaled_cleaned, X_WithDummies, y_cleaned, valid_rows, seed)
        

    # Joining the Categorical and non Categorical features, the last column will be the choice one
    data = pd.concat([X_scaled, X_dummy, y], axis=1)
    data_test = pd.concat([X_test_scaled, X_test_dummy, y_test], axis=1)
    # At this stage, data_test is ready and we'll do nothing more to it

    print('data shape before Filling: ', data.shape)
    data.to_csv("DataPreFilled.csv", index=False)

    match Filltype:
        case 0:
            # drop all rows with NaN entries
            data = data.dropna()
        case 1:
            # Doing a super simple and stupid forward fill.
            # also bfill is possible...
            data = data.ffill()
        case 2:
            # SimpleImputer - most_frequent
            imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
            data2 = pd.DataFrame(imp.fit_transform(data))
            data2.columns = data.columns
            data2.index = data.index
            data = data2
        case 3:
            # This method is quite expensive computationally!
            imputer = IterativeImputer(max_iter=15, random_state = 42, initial_strategy = 'mean')
            data2 = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
            data = data2
        case 4:
            # SimpleImputer - mean
            imp = SimpleImputer(missing_values=np.nan, strategy='mean')
            data2 = pd.DataFrame(imp.fit_transform(data))
            data2.columns = data.columns
            data2.index = data.index
            data = data2
        
        # Attitude questions

    if not use_attitudes:
        DelColumns = []
        DelColumns += [f'Envir{i:02d}' for i in range(1, 7)]
        DelColumns += [f'Mobil{i:02d}' for i in range(1, 28)]
        DelColumns += [f'ResidCh{i:02d}' for i in range(1, 8)]
        DelColumns += [f'LifSty{i:02d}' for i in range(1, 15)]
        data.drop(columns=[col for col in DelColumns if col in data.columns], inplace=True)
        data_test.drop(columns=[col for col in DelColumns if col in data_test.columns], inplace=True)
    

    #extracting the last column (Choice)
    # and converting to NUMPY ARRAYS!
    y = data.iloc[:, -1].values
    data.drop(data.columns[-1], axis=1, inplace = True) # remove the last colum (Choice)
    X_tot = data.values

    y_test = data_test.iloc[:, -1].values
    data_test.drop(data_test.columns[-1], axis=1, inplace = True)
    X_tot_test = data_test.values
    
    print('y shape: ', y.shape)
    print('data shape: ', data.shape)
    print('y_test shape: ', y_test.shape)
    print('data_test shape: ', data_test.shape)
        
    # Augment and split the data
    X_data = X_tot.astype(np.float32)
    y_data = y.astype(int)
    print("X_data shape: ", X_data.shape)
    X_test = X_tot_test.astype(np.float32)
    y_num_test = y_test.astype(int)

    if dataAugmentation:
        X_train, y_train_LOG = augment_data(X_data, y_data, Tsize=0.2)
        print("X_train shape: ", X_train.shape)
        y_train_NN = to_categorical(y_train_LOG, num_classes=2)
        y_test_NN = to_categorical(y_num_test, num_classes=2)
    else:
        X_train = X_data
        y_train_NN = to_categorical(y_data, num_classes=2)
        y_test_NN = to_categorical(y_num_test, num_classes=2)

    model = create_model(X_train, X_test, y_train_NN, y_test_NN, results_show=False)
    # Train the model
    print(X_train.shape, X_test.shape, y_train_NN.shape, y_test_NN.shape)
    history = train_model(model, X_train, X_test, y_train_NN, y_test_NN, epochs=100, batch_size=32, results_show=False)
    y_pred = np.argmax(model.predict(X_test), axis=1)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).tolist()

    if return_data:
        return acc, prec, f1, cm, model, X_test, y_num_test, X_train, y_train_NN, y_test_NN, y_test
    else:
        return acc, prec, f1, cm

# Run All 40 Configs
results = []
configs = list(product(range(5), [True, False], [0, 1], [True, False]))

for i, (fill, aug, cat, att) in enumerate(configs):
    print(f"\nRunning config {i+1}/40 --> Filltype={fill}, Aug={aug}, CatDealer={cat}, Attitudes={att}")
    acc, prec, f1, cm = run_pipeline(fill, aug, cat, att, X_dummy_standard, X_scaled_standard, y_standard, random_state=42)
    results.append({
        'Config_ID': i+1,
        'Filltype': fill,
        'DataAug': aug,
        'CatDealer': cat,
        'AttitudesUsed': att,
        'Accuracy': acc,
        'Precision': prec,
        'F1_Score': f1,
        'Confusion_Matrix': cm
    })

results_df = pd.DataFrame(results)
results_df.to_csv("NN_Comparison_Results.csv", index=False)
print("\n All results saved to 'NN_Comparison_Results.csv'")


In [ ]:

# CONFIG
N_SEEDS = 5
SEED_LIST = [random.randint(0, 99999) for _ in range(N_SEEDS)] 
TestSize = 0.2

# Save seeds for reproducibility
pd.DataFrame({'Seed': SEED_LIST}).to_csv("Used_Seeds.csv", index=False)

# RESULTS STRUCTURE 
configs = list(product(range(5), [True, False], [0, 1], [True, False]))  # 40 combinations
all_results = []

# RUNNING LOOP
for config_id, (Filltype, dataAug, CatDealer, use_attitudes) in enumerate(configs, start=1):
    acc_list = []
    prec_list = []
    f1_list = []

    print(f"\n Config {config_id}/40 - Fill={Filltype}, Aug={dataAug}, Cat={CatDealer}, Att={use_attitudes}")

    for seed in SEED_LIST:

        acc, prec, f1, cm = run_pipeline(fill, aug, cat, att, X_dummy_standard, X_scaled_standard, y_standard, seed)

        acc_list.append(acc)
        prec_list.append(prec)
        f1_list.append(f1)

    result_row = {
        'Config_ID': config_id,
        'Filltype': Filltype,
        'DataAug': dataAug,
        'CatDealer': CatDealer,
        'AttitudesUsed': use_attitudes
    }

    for i in range(N_SEEDS):
        result_row[f'Accuracy_{i+1}'] = acc_list[i]
        result_row[f'Precision_{i+1}'] = prec_list[i]
        result_row[f'F1_Score_{i+1}'] = f1_list[i]

    result_row['Accuracy_Mean'] = np.mean(acc_list)
    result_row['Precision_Mean'] = np.mean(prec_list)
    result_row['F1_Score_Mean'] = np.mean(f1_list)

    all_results.append(result_row)

df_results = pd.DataFrame(all_results)
df_results.to_excel("NN_Comparison_Results_MultiSeed.xlsx", index=False)
print("All results saved to 'NN_Comparison_Results_MultiSeed.xlsx'")
print("Random seeds saved to 'Used_Seeds.csv'")


In [ ]:

# Load the results
df = pd.read_excel("NN_Comparison_Results_MultiSeedF.xlsx")
df.replace(',', '.', regex=True, inplace=True)

# Ensure numeric types
df['Precision_Mean'] = pd.to_numeric(df['Precision_Mean'], errors='coerce')
df['F1_Score_Mean'] = pd.to_numeric(df['F1_Score_Mean'], errors='coerce')
df['Accuracy_Mean'] = pd.to_numeric(df['Accuracy_Mean'], errors='coerce')

# Filter for high precision average > 0.925
high_precision_df = df[df['Precision_Mean'] > 0.925].copy()

# Identify the best config based on F1 Score Mean
best_config_idx = high_precision_df['F1_Score_Mean'].idxmax()
best_config = high_precision_df.loc[best_config_idx]

# Plotting
plt.figure(figsize=(10, 6))
plt.grid(True, linestyle='--', alpha=0.6)

plt.scatter(
    high_precision_df['Accuracy_Mean'],
    high_precision_df['F1_Score_Mean'],
    c='royalblue',
    s=80,
    edgecolor='black',
    label='High-Precision Configs (Precision > 0.925)'
)

# Labels for not overlapping
label_styles = {
    8: {'xytext': (25, 10), 'ha': 'left'},
    32: {'xytext': (-30, -10), 'ha': 'right'},
    22: {'xytext': (-20, 10), 'ha': 'right'},
    40: {'xytext': (5, 25), 'ha': 'left'},
    23: {'xytext': (5, 25), 'ha': 'left'},
}

# Annotate each config with ID (except best)
for _, row in high_precision_df.iterrows():
    cfg_id = int(row['Config_ID'])
    if cfg_id != int(best_config['Config_ID']):
        style = label_styles.get(cfg_id, {'xytext': (10, 10), 'ha': 'left'})
        plt.annotate(
            f"ID {cfg_id}",
            (row['Accuracy_Mean'], row['F1_Score_Mean']),
            textcoords="offset points",
            xytext=style['xytext'],
            ha=style['ha'],
            fontsize=9,
            arrowprops=dict(
                arrowstyle='->',
                color='gray',
                connectionstyle='arc3,rad=0.2'
            )
        )

# Highlight best config
plt.scatter(
    best_config['Accuracy_Mean'],
    best_config['F1_Score_Mean'],
    c='darkorange',
    s=120,
    edgecolor='black',
    label=f"Best Config (ID {int(best_config['Config_ID'])})",
    marker='*',
    zorder=5
)

# Annotate best config
plt.annotate(
    f"Best Config\nID: {int(best_config['Config_ID'])}\nF1: {best_config['F1_Score_Mean']:.3f}",
    (best_config['Accuracy_Mean'], best_config['F1_Score_Mean']),
    textcoords="offset points",
    xytext=(10, -25),
    ha='left',
    fontsize=10,
    arrowprops=dict(
        arrowstyle='->',
        color='gray',
        connectionstyle='arc3,rad=-0.2'
    )
)

# Labels and title
plt.xlabel('Mean Accuracy', fontsize=13)
plt.ylabel('Mean F1 Score', fontsize=13)
#plt.title('Best Configurations with Precision_Mean > 0.925', fontsize=15)
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:

# FIXED CONFIG ID 23 
Filltype = 2
dataAug = False
CatDealer = 1
AttitudesUsed = True
TestSize = 0.2
SEEDS = [random.randint(0, 100000) for _ in range(5)]

# Define NN Architectures 
architectures = [
    {'name': '1.1',      'arch': [32],              'act': 'relu',    'final_act': 'softmax', 'batch': 32},
    {'name': '1.2',      'arch': [32],              'act': 'relu',    'final_act': 'softmax', 'batch': 44},
    {'name': '1.3',      'arch': [32],              'act': 'relu',    'final_act': 'softmax', 'batch': 64},
    {'name': '2.1', 'arch': [64, 32],          'act': 'relu',    'final_act': 'softmax', 'batch': 32},
    {'name': '2.2', 'arch': [64, 32],          'act': 'relu',    'final_act': 'softmax', 'batch': 44},
    {'name': '2.3', 'arch': [64, 32],          'act': 'relu',    'final_act': 'softmax', 'batch': 64},
    {'name': '3.1', 'arch': [64, 32, 16],      'act': 'relu',    'final_act': 'softmax', 'batch': 32},
    {'name': '3.2 - Baseline', 'arch': [64, 32, 16],      'act': 'relu',    'final_act': 'softmax', 'batch': 44},
    {'name': '3.3', 'arch': [64, 32, 16],      'act': 'relu',    'final_act': 'softmax', 'batch': 64},
    {'name': '4.1',      'arch': [128, 64, 32, 16],      'act': 'relu',    'final_act': 'softmax', 'batch': 32},
    {'name': '4.2',      'arch': [128, 64, 32, 16],      'act': 'relu',    'final_act': 'softmax', 'batch': 44},
    {'name': '4.3',      'arch': [128, 64, 32, 16],      'act': 'relu',    'final_act': 'softmax', 'batch': 64},
    {'name': '5.1',   'arch': [64, 32, 16, 32, 64, 32, 16],      'act': 'relu', 'final_act': 'softmax', 'batch': 32},
    {'name': '5.2',   'arch': [64, 32, 16, 32, 64, 32, 16],      'act': 'relu', 'final_act': 'softmax', 'batch': 44},
    {'name': '5.3',   'arch': [64, 32, 16, 32, 64, 32, 16],      'act': 'relu', 'final_act': 'softmax', 'batch': 64},
    {'name': '6.1',   'arch': [128, 64, 32, 16, 32, 64, 128, 64, 32, 16],      'act': 'relu', 'final_act': 'softmax', 'batch': 32},
    {'name': '6.2',   'arch': [128, 64, 32, 16, 32, 64, 128, 64, 32, 16],      'act': 'relu', 'final_act': 'softmax', 'batch': 44},
    {'name': '6.3',   'arch': [128, 64, 32, 16, 32, 64, 128, 64, 32, 16],      'act': 'relu', 'final_act': 'softmax', 'batch': 64}
]


# Model builder
def build_model(arch, input_shape, activation, final_activation):
    model = Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    for units in arch:
        model.add(layers.Dense(units, activation=activation))
    model.add(layers.Dense(2, activation=final_activation))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Store Results
results = []

for arch_def in architectures:
    acc_list, prec_list, f1_list = [], [], []
    print(f"\n Testing {arch_def['name']} ...")

    for seed in SEEDS:
        RandomState = seed
        # Get fresh data each run
        _, _, _, _, _, X_test, _, X_train, y_train_NN, y_test_NN, y_test = run_pipeline(
            Filltype, dataAug, CatDealer, AttitudesUsed, X_dummy_standard, X_scaled_standard, y_standard, RandomState, return_data=True
        )

        # Build model
        model = build_model(
            arch=arch_def['arch'],
            input_shape=X_train.shape[1],
            activation=arch_def['act'],
            final_activation=arch_def['final_act']
        )

        # Train the model
        train_model(
            model,
            X_train,
            X_test,
            y_train_NN,
            y_test_NN,
            epochs=100,
            batch_size=arch_def['batch'],
            results_show=True
        )

        # Evaluate
        y_pred = np.argmax(model.predict(X_test), axis=1)

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred).tolist()

        acc_list.append(acc)
        prec_list.append(prec)
        f1_list.append(f1)

    results.append({
        'Architecture': arch_def['name'],
        'Hidden_Layers': str(arch_def['arch']),
        'Activation': arch_def['act'],
        'BatchSize': arch_def['batch'],
        'Accuracy_Mean': np.mean(acc_list),
        'Precision_Mean': np.mean(prec_list),
        'F1_Score_Mean': np.mean(f1_list),
        'Acc_Seeds': acc_list,
        'Prec_Seeds': prec_list,
        'F1_Seeds': f1_list,
        'Seeds': SEEDS
    })

df = pd.DataFrame(results)
df.to_excel("NN_Architecture_Search_Final.xlsx", index=False)
print("\n Results saved to 'NN_Architecture_Search_Final.xlsx'")


In [ ]:
#Train the final model

# FIXED CONFIG ID 23 
Filltype = 2
dataAug = False
CatDealer = 1
AttitudesUsed = True
TestSize = 0.2
seed = 12345  # Fixed seed for reproducibility

_, _, _, _, _, X_test, _, X_train, y_train_NN, y_test_NN, y_test = run_pipeline(
Filltype, dataAug, CatDealer, AttitudesUsed, X_dummy_standard, X_scaled_standard, y_standard, RandomState, return_data=True
)

# Define NN Architecture (final model - 3.2)

final_model = build_model(
    arch=[64, 32, 16],
    input_shape=X_train.shape[1],
    activation='relu',
    final_activation='softmax'
)

history = train_model(
            final_model,
            X_train,
            X_test,
            y_train_NN,
            y_test_NN,
            epochs=100,
            batch_size=44,
            results_show=True
        )

final_model.save("Final_Model.keras")

plot_history(history)

# Evaluate
y_pred = np.argmax(model.predict(X_test), axis=1)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred).tolist()

print("Final Model Evaluation:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(cm)


In [ ]:
#Global Sensitivity Analysis

# Load full dataset for ground truth computation
raw_data = pd.read_csv("ModeChoiceOptima.txt", sep="\t", header=None)
featureNames = raw_data.iloc[0]
raw_data.columns = featureNames
raw_data = raw_data[1:]

nan_mask = X_dummy_standard.isna().any(axis=1)  # True for rows with any NaN
# Drop those rows in both X_dummy, X_scaled and y
X_dummy_cleaned = X_dummy_standard[~nan_mask].reset_index(drop=True)
X_scaled_cleaned = X_scaled_standard[~nan_mask].reset_index(drop=True)
y_cleaned = y[~nan_mask].reset_index(drop=True)

X_WithDummies = dummy_inator(X_dummy_cleaned, False)
X_input = pd.concat([X_scaled_cleaned, X_WithDummies], axis=1)

X_input = X_input.dropna().astype(int)

pt_count = np.sum(y_cleaned == 0)
priv_count = np.sum(y_cleaned == 1)
total_valid = pt_count + priv_count
true_ground_pct = pt_count / total_valid * 100
print(f"Ground Truth from dataset: {pt_count}/{total_valid} → {true_ground_pct:.2f}% use PT")


final_model = load_model("Final_Model.keras")


y_pred_base = np.argmax(final_model.predict(X_input, verbose=0), axis=1)
pt_pred_base_pct = np.sum(y_pred_base == 0) / len(y_pred_base) * 100
print(f"Model Baseline Prediction: {pt_pred_base_pct:.2f}% use PT")

X_notscaled = data.drop(columns=['Choice'])
X_notscaled = X_notscaled.reset_index(drop=True)
X_notscaled_cleaned = X_notscaled[~nan_mask].reset_index(drop=True)

X_tovary = pd.concat([X_notscaled_cleaned, X_WithDummies], axis=1)
X_tovary = X_tovary.dropna().astype(int)

X_notscaled_tovary = X_tovary.iloc[:, :99]
X_notscaled_dummies = X_tovary.iloc[:, 99:]

# Features to vary (individually)
features_to_vary = ["NbTransf", "TimePT", "WalkingTimePT", "WaitingTimePT", "CostPT"]
variation_percentages = list(range(-50, 51, 1))  # 1% steps
sensitivity_results = {}

# Individual feature variation
for feat in features_to_vary:
    if feat not in X_notscaled_cleaned.columns:
        print(f" Skipping {feat}, not found.")
        continue

    print(f"Varying {feat}")
    original = X_notscaled_cleaned[feat].copy()
    results = []

    for pct in variation_percentages:
        factor = 1 + pct / 100
        X_mod = X_notscaled_cleaned.copy()
        X_mod[feat] = original * factor

        temp_scaled_mod = scaler.transform(X_mod)
        X_mod_scaled = pd.DataFrame(temp_scaled_mod, columns=X_mod.columns)
        X_mod_input = pd.concat([X_mod_scaled, X_WithDummies], axis=1)
        X_mod_input = X_mod_input.dropna().astype(int)

        y_pred = np.argmax(final_model.predict(X_mod_input, verbose=0), axis=1)
        pct_pt_users = np.sum(y_pred == 0) / len(y_pred) * 100  
        results.append(pct_pt_users)

    sensitivity_results[feat] = results

# Combined improvement scenario
combined_results = []
good_directions = {
    "NbTransf": 1,
    "TimePT": 1,
    "WalkingTimePT": 1,
    "WaitingTimePT": 1,
    "CostPT": 1
}

for pct in variation_percentages:
    X_comb = X_notscaled_cleaned.copy()
    for feat, direction in good_directions.items():
        if feat in X_comb.columns:
            X_comb[feat] *= (1 + direction * pct / 100)

    temp_scaled_comb = scaler.transform(X_comb)
    X_comb_scaled = pd.DataFrame(temp_scaled_comb, columns=X_comb.columns)
    X_comb_input = pd.concat([X_comb_scaled, X_WithDummies], axis=1)
    X_comb_input = X_comb_input.dropna().astype(int)
  
    y_pred = np.argmax(final_model.predict(X_comb_input, verbose=0), axis=1)
    pct_pt_users = np.sum(y_pred == 0) / len(y_pred) * 100
    combined_results.append(pct_pt_users)

# Plotting
all_features = list(sensitivity_results.keys()) + ["Combined"]
ncols = 3
nrows = (len(all_features) + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(16, 4 * nrows))
axes = axes.flatten()

for idx, feat in enumerate(all_features):
    ax = axes[idx]
    values = sensitivity_results[feat] if feat != "Combined" else combined_results

    ax.plot(variation_percentages, values, marker='.', linestyle='-', color='tab:blue', label='Predicted % PT')
    ax.axhline(y=true_ground_pct, linestyle='--', color='gray', label='True % from Dataset')
    ax.axhline(y=pt_pred_base_pct, linestyle=':', color='green', label='Model Baseline %')
    ax.set_title(f"Effect of Varying {feat}", fontsize=13)
    ax.set_xlabel("Percentage Change", fontsize=11)
    ax.set_ylabel("% Predicted PT Users", fontsize=11)
    ax.grid(True, linestyle='--', alpha=0.6)

    # Set custom x-ticks every 5%
    ax.set_xticks([i for i in variation_percentages if i % 5 == 0])

    # Zoom in on the relevant Y range
    y_min = min(values + [true_ground_pct, pt_pred_base_pct])
    y_max = max(values + [true_ground_pct, pt_pred_base_pct])
    padding = max(0.3, (y_max - y_min) * 0.2)
    ax.set_ylim([y_min - padding, y_max + padding])

    ax.legend()

# Remove unused subplots
for i in range(len(all_features), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
# Regional Sensitivity Analysis
region_columns = [col for col in X_input.columns if col.startswith('Region_')]
pt_percentage_per_region = {}

for col in region_columns:
    # Get the indexes where this region column is 1
    region_idx = X_input[X_input[col] == 1].index
    region_y = y_cleaned.loc[region_idx]

    pt_count = np.sum(region_y == 0)
    priv_count = np.sum(region_y == 1)
    total_valid = pt_count + priv_count

    if total_valid == 0:
        continue

    pt_pct = pt_count / total_valid * 100
    pt_percentage_per_region[col] = pt_pct

    print(f"{col}: {pt_count}/{total_valid} → {pt_pct:.2f}% use PT")

# Load model
final_model = load_model("Final_Model.keras")

# Define settings
features_to_vary = ["NbTransf", "TimePT", "WalkingTimePT", "WaitingTimePT", "CostPT"]
variation_percentages = list(range(-50, 51, 1))
region_columns = [col for col in X_input.columns if col.startswith("Region_")]

X_notscaled_cleaned = X_notscaled_cleaned.copy()

# Store all region-specific data
region_sensitivity = {}

for region_col in region_columns:
    print(f"Processing {region_col}")
    
    # Get data and labels for this region
    region_idx = X_input[X_input[region_col] == 1].index
    region_data = X_input.loc[region_idx]
    region_y = y_cleaned.loc[region_idx]

    X_region_notscaled_idx = X_tovary[X_tovary[region_col] == 1].index
    X_region_notscaled = X_notscaled_cleaned.loc[X_region_notscaled_idx]
    region_dummies = X_notscaled_dummies.loc[X_region_notscaled_idx]
    
    if len(region_data) == 0:
        continue

    # Ground truth % PT
    pt_count = np.sum(region_y == 0)
    priv_count = np.sum(region_y == 1)
    total_valid = pt_count + priv_count
    if total_valid == 0:
        continue
    true_ground_pct = pt_count / total_valid * 100

    # Model baseline prediction
    y_pred_base = np.argmax(final_model.predict(region_data, verbose=0), axis=1)
    pt_pred_base_pct = np.sum(y_pred_base == 0) / len(y_pred_base) * 100

    region_result = {}

    for feat in features_to_vary:
        if feat not in X_region_notscaled.columns:
            print(f" Skipping {feat}, not found.")
            continue

        original = X_region_notscaled[feat].copy()
        result = []

        for pct in variation_percentages:
            factor = 1 + pct / 100
            X_mod = X_region_notscaled.copy()
            X_mod[feat] = original * factor

            temp_scaled_regions = scaler.transform(X_mod)
            X_regions_scaled = pd.DataFrame(temp_scaled_regions, columns=X_mod.columns)
            X_regions_scaled = X_regions_scaled.reset_index(drop=True)
            region_dummies = region_dummies.reset_index(drop=True)
            X_regions_input = pd.concat([X_regions_scaled, region_dummies], axis=1)
            X_regions_input = X_regions_input.dropna().astype(int)

            y_pred = np.argmax(final_model.predict(X_regions_input, verbose=0), axis=1)
            pct_pt_users = np.sum(y_pred == 0) / len(y_pred) * 100
            result.append(pct_pt_users)

        region_result[feat] = result

    # Save everything
    region_sensitivity[region_col] = {
        "sensitivity": region_result,
        "true_pct": true_ground_pct,
        "baseline_pct": pt_pred_base_pct
    }

# Plotting
ncols = 2
nrows = (len(region_sensitivity) + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(16, 4 * nrows))
axes = axes.flatten()

for idx, (region, result) in enumerate(region_sensitivity.items()):
    ax = axes[idx]

    for feat in features_to_vary:
        if feat in result["sensitivity"]:
            ax.plot(variation_percentages, result["sensitivity"][feat], label=feat)

    ax.axhline(y=result["true_pct"], linestyle='--', color='gray', label='True % PT')
    ax.axhline(y=result["baseline_pct"], linestyle=':', color='green', label='Model Baseline %')
    ax.set_title(f"{region.replace('_', ' ')}", fontsize=13)
    ax.set_xlabel("Percentage Change", fontsize=11)
    ax.set_ylabel("% Predicted PT Users", fontsize=11)
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.set_xticks([i for i in variation_percentages if i % 10 == 0])
    ax.legend()

# Remove unused subplots
for i in range(len(region_sensitivity), len(axes)):
    fig.delaxes(axes[i])

fig.suptitle("Regional Sensitivity Analysis: % Predicted PT Users vs Feature Variation", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()



In [ ]:
# Ticket Sensitivity Analysis

# Define tickets and their corresponding scaled values
ticket_info = {
    'HalfFareST': -1,
    'LineRelST': -3,
    'GenAbST': -3,
    'AreaRelST': -3,
    'OtherST': -4
}
# Tickets to evaluate
ticket_cols = list(ticket_info.keys())
all_cols = ['Baseline'] + ticket_cols


region_columns = [col for col in X_input.columns if col.startswith("Region_")]

# Baseline prediction (global)
y_pred_base_global = np.argmax(final_model.predict(X_input, verbose=0), axis=1)
pt_pct_base_global = np.sum(y_pred_base_global == 0) / len(y_pred_base_global) * 100

# Store all results
ticket_results = {}
ticket_colors = {
    'Baseline': 'tab:blue',
    'HalfFareST': 'tab:orange',
    'LineRelST': 'tab:green',
    'GenAbST': 'tab:red',
    'AreaRelST': 'tab:purple',
    'OtherST': 'tab:brown'
}
# All regions + global
regions_to_plot = ['GLOBAL'] + list(region_columns)
region_labels = [r.replace('_', ' ') for r in regions_to_plot]

# Collect values per region
data_matrix = []

for region in regions_to_plot:
    region_values = []

    # Baseline first
    if region == 'GLOBAL':
        baseline = pt_pct_base_global
    else:
        # Use any ticket's baseline since it's the same across all
        sample_ticket = next(iter(ticket_results))
        baseline = ticket_results[sample_ticket]['regional'][region]['Before']

    region_values.append(baseline)

    # Then each ticket's effect
    for ticket in ticket_cols:
        if region == 'GLOBAL':
            pt_pct = ticket_results[ticket]['global_after']
        else:
            pt_pct = ticket_results[ticket]['regional'].get(region, {}).get('After', np.nan)
        region_values.append(pt_pct)

    data_matrix.append(region_values)

data_matrix = np.array(data_matrix)  

# Plotting
n_regions = len(regions_to_plot)
n_bars = len(all_cols)
x = np.arange(n_regions)
bar_width = 0.12

fig, ax = plt.subplots(figsize=(16, 6))

for i, label in enumerate(all_cols):
    offset = (i - n_bars / 2) * bar_width + bar_width / 2
    ax.bar(x + offset, data_matrix[:, i], width=bar_width, label=label, color=ticket_colors[label])

ax.set_ylabel('% Predicted PT Users')
ax.set_title('Impact of Season Tickets on Public Transport Usage per Region')
ax.set_xticks(x)
ax.set_xticklabels(region_labels, rotation=45)
ax.legend(title='Scenario')
ax.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()


In [ ]:
# Distribution of Season Tickets per Region

ticket_cols = ['HalfFareST', 'LineRelST', 'GenAbST', 'AreaRelST', 'OtherST']
ticket_colors = {
    'HalfFareST': 'tab:orange',
    'LineRelST': 'tab:green',
    'GenAbST': 'tab:red',
    'AreaRelST': 'tab:purple',
    'OtherST': 'tab:brown'
}
region_columns = [col for col in X_input.columns if col.startswith("Region_")]

# List of regions + global
regions_to_plot = ['GLOBAL'] + region_columns
region_labels = [r.replace('_', ' ') for r in regions_to_plot]

# Data matrix: rows = regions, columns = ticket types
data_matrix = []

for region in regions_to_plot:
    if region == 'GLOBAL':
        region_data = X_input
    else:
        region_data = X_input[X_input[region] == 1]

    region_row = []
    for ticket in ticket_cols:
        if ticket in region_data.columns:
            count_with_ticket = (region_data[ticket] != 0).sum()
            pct_with_ticket = count_with_ticket / len(region_data) * 100 if len(region_data) > 0 else 0
        else:
            pct_with_ticket = np.nan
        region_row.append(pct_with_ticket)

    data_matrix.append(region_row)

data_matrix = np.array(data_matrix)

# Plotting
n_regions = len(regions_to_plot)
n_tickets = len(ticket_cols)
x = np.arange(n_regions)
bar_width = 0.15

fig, ax = plt.subplots(figsize=(16, 6))

for i, ticket in enumerate(ticket_cols):
    offset = (i - n_tickets / 2) * bar_width + bar_width / 2
    ax.bar(
        x + offset,
        data_matrix[:, i],
        width=bar_width,
        label=ticket,
        color=ticket_colors[ticket]
    )

ax.set_ylabel('% of Population with Season Ticket')
ax.set_title('Original Distribution of Season Tickets per Region')
ax.set_xticks(x)
ax.set_xticklabels(region_labels, rotation=45)
ax.legend(title='Ticket Type')
ax.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()


In [ ]:
# CostCarCHF Sensitivity Analysis

# Load model
final_model = load_model("Final_Model.keras")

# Feature to vary
feature_to_vary = "CostCarCHF"
variation_percentages = list(range(-100, 101, 1))  # from -100% to +100%

# Baseline prediction
y_pred_base = np.argmax(final_model.predict(X_input, verbose=0), axis=1)
pt_pred_base_pct = np.sum(y_pred_base == 0) / len(y_pred_base) * 100
print(f"Model Baseline Prediction: {pt_pred_base_pct:.2f}% use PT")

# Ground truth
pt_count = np.sum(y_cleaned == 0)
priv_count = np.sum(y_cleaned == 1)
total_valid = pt_count + priv_count
true_ground_pct = pt_count / total_valid * 100
print(f"Ground Truth from dataset: {pt_count}/{total_valid} → {true_ground_pct:.2f}% use PT")

# Sensitivity analysis for CostCarCHF
print(f"\n Varying {feature_to_vary}")
original = X_notscaled_cleaned[feature_to_vary].copy()
pt_pct_results = []

for pct in variation_percentages:
    factor = 1 + pct / 100
    X_mod = X_notscaled_cleaned.copy()
    X_mod[feature_to_vary] = original * factor

    temp_scaled_mod = scaler.transform(X_mod)
    X_mod_scaled = pd.DataFrame(temp_scaled_mod, columns=X_mod.columns)
    X_mod_input = pd.concat([X_mod_scaled, X_WithDummies], axis=1)
    X_mod_input = X_mod_input.dropna().astype(int)
    
    y_pred = np.argmax(final_model.predict(X_mod_input, verbose=0), axis=1)
    pct_pt_users = np.sum(y_pred == 0) / len(y_pred) * 100
    pt_pct_results.append(pct_pt_users)

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(variation_percentages, pt_pct_results, marker='o', linestyle='-', color='tab:blue', label='Predicted % PT')
plt.axhline(y=true_ground_pct, linestyle='--', color='gray', label='True % from Dataset')
plt.axhline(y=pt_pred_base_pct, linestyle=':', color='green', label='Model Baseline %')
plt.title("Effect of Varying CostCarCHF on PT Usage")
plt.xlabel("Percentage Change in CostCarCHF")
plt.ylabel("% Predicted PT Users")
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks([i for i in variation_percentages if i % 5 == 0])
plt.legend()
plt.tight_layout()
plt.show()
